<a href="https://colab.research.google.com/github/jiunyu-hsieh/supermarket-marketing-analytics/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Prepared Data**

In [12]:
# Mounting google drive on google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Print work directory (pwd)
%cd /content/drive/MyDrive/Colab Notebooks/4 Analytics-enabled Marketing

/content/drive/MyDrive/Colab Notebooks/4 Analytics-enabled Marketing


In [14]:
# Import library
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

In [15]:
# Import dataset of 10% of loyatly customers
dataset=pd.read_excel("data preparation.xlsx")

In [16]:
# Show count of rows and columns
dataset.shape
# Show the first 5 rows
dataset.head(5)

,DemAffl,DemAge,DemClusterGroup,DemGender,DemReg,DemTVReg,LoyalClass,LoyalSpend,LoyalTime,TargetBuy
0,5,51,2,0,3,3,3,0.01,5,0
1,15,51,2,1,3,3,0,8000.00,5,1
2,8,51,2,1,0,2,3,0.01,5,1
3,8,28,2,0,3,3,3,0.01,5,1
4,14,67,2,0,3,3,3,0.01,7,0


# **Checking for Multicollinearity (VIF)**


In [21]:
# VIF Function 
from statsmodels.stats.outliers_influence import variance_inflation_factor

def cal_vif(x):
  vif=pd.DataFrame()
  vif["variables"]=x.columns
  vif["VIF"]=[variance_inflation_factor(x.values,i) for i in range(x.shape[1])]

  return(vif)

In [22]:
# Calculate VIF
x=dataset.iloc[:,0:9]
cal_vif(x)

,variables,VIF
0,DemAffl,6.278442
1,DemAge,10.735119
2,DemClusterGroup,3.659693
3,DemGender,1.435469
4,DemReg,2.474606
5,DemTVReg,3.752267
6,LoyalClass,3.851180
7,LoyalSpend,1.863146
8,LoyalTime,3.138948


In [23]:
# Dependent value and independent value
y=dataset.iloc[:,9].values
x=dataset.iloc[:,0:9].values

In [24]:
# Spit dataset into trainning data and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

# **Modeling**

In [25]:
classifer=LogisticRegression(max_iter=200)
classifer.fit(x_train,y_train)
y_pred=classifer.predict(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Export logistic regression classifer to later use in prediction
import joblib
joblib.dump(classifer,'./Classifer_LoyalCustomers')

['./Classifer_LoyalCustomers']

In [ ]:
print(confusion_matrix(y_test,y_pred))

[[3234  133]
 [ 729  349]]


In [ ]:
print(accuracy_score(y_test,y_pred))

0.80607424071991


In [ ]:
# Prediction of buy or not
predictions = classifer.predict_proba(x_test)
predictions

array([[0.8243508 , 0.1756492 ],
       [0.6923718 , 0.3076282 ],
       [0.53103197, 0.46896803],
       ...,
       [0.83973293, 0.16026707],
       [0.88258438, 0.11741562],
       [0.91709338, 0.08290662]])

In [ ]:
# Output files
df_predictopn_prob=pd.DataFrame(predictions,columns=['prob_0','prob_1'])
df_actual_outcome=pd.DataFrame(y_test,columns=['actual_outcome'])
df_x_test=pd.DataFrame(x_test)

df_output=pd.concat([df_x_test, df_actual_outcome, df_predictopn_prob],axis=1) 

df_output.to_excel('10Percent Output.xlsx')

df_output.head(5)


,0,1,2,3,4,5,6,7,8,actual_outcome,prob_0,prob_1
0,10.0,58.0,3.0,1.0,3.0,8.0,0.0,12632.66,4.0,0,0.824351,0.175649
1,10.0,51.0,2.0,0.0,3.0,8.0,0.0,6000.00,6.0,0,0.692372,0.307628
2,16.0,65.0,1.0,0.0,2.0,6.0,0.0,6053.35,7.0,0,0.531032,0.468968
3,5.0,60.0,3.0,0.0,3.0,3.0,2.0,5000.00,1.0,0,0.935008,0.064992
4,9.0,52.0,3.0,0.0,0.0,4.0,2.0,3500.00,6.0,0,0.755145,0.244855
